In [1]:
#Import semua framework dan library yang dibutuhkan
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import matplotlib.pyplot as plt
import pathlib
import shutil
import os


# **Memindahkan folder ke working**

In [2]:
path_folder_input = '/kaggle/input/tilapia-fish-disease/dataset'
path_working = '/kaggle/working/'
shutil.move(path_folder_input, path_working)

#cek folder sudah pindah atau belum
path_folder_tujuan = os.path.join(path_working, 'dataset')  
if os.path.exists(path_folder_tujuan):
    print(f"Folder berhasil dipindahkan ke '{path_working}'")
else:
    print("Gagal memindahkan folder")

OSError: [Errno 30] Read-only file system: 'IMG20231122115244.jpg'

In [3]:
nama_folder_baru = 'validation'
path_folder_baru = os.path.join('/kaggle/working/dataset', nama_folder_baru)
os.makedirs(path_folder_baru)

nama_folder_baru = 'aeromonas_infection'
path_folder_baru = os.path.join('/kaggle/working/dataset/validation', nama_folder_baru)
os.makedirs(path_folder_baru)

nama_folder_baru = 'sehat'
path_folder_baru = os.path.join('/kaggle/working/dataset/validation', nama_folder_baru)
os.makedirs(path_folder_baru)

nama_folder_baru = 'white_spot'
path_folder_baru = os.path.join('/kaggle/working/dataset/validation', nama_folder_baru)
os.makedirs(path_folder_baru)

# Melakukan Data Split Sebanyak 20% untuk Validasi

In [4]:
#randomize data dari training lalu pindahkan 20%nya ke validation
import random
# Fungsi
def randomize_move_files(folder_sumber, folder_tujuan, persentase_pindah):
    files = os.listdir(folder_sumber)
    
    jumlah_file_diambil = int(persentase_pindah * len(files))
    
    files_yang_diambil = random.sample(files, jumlah_file_diambil)

    # Pindahkan file-file yang terpilih ke folder tujuan
    for file in files_yang_diambil:
        path_file_sumber = os.path.join(folder_sumber, file)
        path_file_tujuan = os.path.join(folder_tujuan, file)
        shutil.move(path_file_sumber, path_file_tujuan)
        
    print(f"{jumlah_file_diambil} file telah dipindahkan ke '{folder_tujuan}'")

#pindahkan data ikan terinfeksi aeromonas ke validasi 20%
folder_sumber = '/kaggle/working/dataset/training/aeromonas_infection'
folder_tujuan = '/kaggle/working/dataset/validation/aeromonas_infection'
jumlah = 0.2
randomize_move_files(folder_sumber, folder_tujuan, jumlah)

#pindahkan data ikan terinfeksi aeromonas ke validasi 20%
folder_sumber = '/kaggle/working/dataset/training/white_spot'
folder_tujuan = '/kaggle/working/dataset/validation/white_spot'
jumlah = 0.2
randomize_move_files(folder_sumber, folder_tujuan, jumlah)

#pindahkan data ikan sehat ke validasi 20%
folder_sumber = '/kaggle/working/dataset/training/sehat'
folder_tujuan = '/kaggle/working/dataset/validation/sehat'
jumlah = 0.2
randomize_move_files(folder_sumber, folder_tujuan, jumlah)
        
    

113 file telah dipindahkan ke '/kaggle/working/dataset/validation/aeromonas_infection'
104 file telah dipindahkan ke '/kaggle/working/dataset/validation/white_spot'
104 file telah dipindahkan ke '/kaggle/working/dataset/validation/sehat'


# Labeling dan Augmentasi Data

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAINING_DIR = "/kaggle/working/dataset/training"
training_datagen = ImageDataGenerator(
      rescale = 1./255,
	  rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      vertical_flip=True,
      fill_mode='nearest')

VALIDATION_DIR = "/kaggle/working/dataset/validation"
validation_datagen = ImageDataGenerator(rescale = 1/255)

train_generator = training_datagen.flow_from_directory(
    TRAINING_DIR,
    target_size=(300,300),
    class_mode='categorical',
    batch_size=20
)

validation_generator = validation_datagen.flow_from_directory(
    VALIDATION_DIR,
    target_size=(300,300),
    class_mode='categorical',
    batch_size=10
)

Found 1287 images belonging to 3 classes.
Found 321 images belonging to 3 classes.


# Penggunaan Pre-trained Model InceptionV3 

In [6]:
import urllib.request
from urllib.request import urlopen
from tensorflow.keras.applications.inception_v3 import InceptionV3
    
inceptionv3 = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
urllib.request.urlretrieve(inceptionv3, 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5')
local_weights_file = 'inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

pre_trained_model =  InceptionV3(input_shape = (300, 300, 3),
                                  include_top = False,
                                  weights = None)
pre_trained_model.load_weights(local_weights_file)

for layer in pre_trained_model.layers:
    layer.trainable = False

last_layer =   pre_trained_model.get_layer('mixed7')
last_output = last_layer.output

In [7]:
# Mengatur Ouput Layer 
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)
x = layers.Dense(3, activation='softmax')(x)

model = Model(pre_trained_model.input, x)

# **Pelatihan Model**

In [8]:
model.compile(optimizer=RMSprop(learning_rate=0.001),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

history = model.fit(train_generator,
                    validation_data = validation_generator,
                    steps_per_epoch = 39,
                    validation_steps = 30,
                    epochs = 20,
                    verbose = 2
                    )

Epoch 1/20
39/39 - 215s - loss: 14.7061 - accuracy: 0.5984 - val_loss: 2.0210 - val_accuracy: 0.8000 - 215s/epoch - 6s/step
Epoch 2/20
39/39 - 210s - loss: 1.6338 - accuracy: 0.7628 - val_loss: 1.9615 - val_accuracy: 0.7167 - 210s/epoch - 5s/step
Epoch 3/20
39/39 - 210s - loss: 0.6802 - accuracy: 0.8026 - val_loss: 0.3716 - val_accuracy: 0.8833 - 210s/epoch - 5s/step
Epoch 4/20
39/39 - 213s - loss: 0.5662 - accuracy: 0.8359 - val_loss: 0.9653 - val_accuracy: 0.7433 - 213s/epoch - 5s/step
Epoch 5/20
39/39 - 208s - loss: 0.4032 - accuracy: 0.8605 - val_loss: 0.3898 - val_accuracy: 0.9200 - 208s/epoch - 5s/step
Epoch 6/20
39/39 - 209s - loss: 0.4822 - accuracy: 0.8538 - val_loss: 0.2682 - val_accuracy: 0.9267 - 209s/epoch - 5s/step
Epoch 7/20
39/39 - 208s - loss: 0.3111 - accuracy: 0.8879 - val_loss: 0.2349 - val_accuracy: 0.9367 - 208s/epoch - 5s/step
Epoch 8/20
39/39 - 209s - loss: 0.3021 - accuracy: 0.9000 - val_loss: 0.4924 - val_accuracy: 0.8600 - 209s/epoch - 5s/step
Epoch 9/20
39/3

# Menyimpan model dalam format .h5

In [9]:
nama_folder_baru = 'saved_model'
path_folder_baru = os.path.join('/kaggle/working/', nama_folder_baru)
os.makedirs(path_folder_baru)

export_dir = '/kaggle/working/saved_model'
tf.saved_model.save(model, export_dir)

model.save("model.h5")

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


# Konversi Model ke format tflite

In [10]:
converter = tf.lite.TFLiteConverter.from_saved_model(export_dir)
tflite_model = converter.convert()

tflite_model_file = pathlib.Path('tilapia_disease_detec.tflite')
tflite_model_file.write_bytes(tflite_model)

944995944

In [11]:
from IPython.display import FileLink

FileLink(r'model.h5')

/kaggle/working/model.h5

In [12]:
FileLink(r'tilapia_disease_detec.tflite')

/kaggle/working/tilapia_disease_detec.tflite